In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install pyspark pyarrow

     |████████████████████████████████| 281.3 MB 34 kB/s 
     |████████████████████████████████| 198 kB 62.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805911 sha256=e66eecffb375e81a3fae5726e37809037af2a74da92dccdde03c9f13a0c975ab
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
    .master("local[2]")\
    .appName("Lesson_2")\
    .config("spark.executor.instances",2)\
    .config("spark.executor.memory",'2g')\
    .config("spark.executor.cores",1)\
    .getOrCreate()
sc = spark.sparkContext

# Самостоятельная работа к уроку 4
На уроке мы попробовали оконные и пользовательские функции. Теперь закрепим полученные знания.

## Данные: [google drive: raw_sales.csv](https://drive.google.com/file/d/1G2N7Mnt4-Tqz4JdJxutGDMbJiOr32kZp/view?usp=sharing)

 Каждая строчка это продажа жилья, которая состоит из следующих полей (думаю описание не требуется):
*   date of sale
*   price
*   property type
*   number of bedrooms
*   4digit postcode

In [32]:
#чтение данных 
df = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/Spark/raw_sales.csv', header=True, inferSchema=True)
df.printSchema()
df.show(truncate=False)

root
 |-- datesold: string (nullable = true)
 |-- postcode: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- propertyType: string (nullable = true)
 |-- bedrooms: integer (nullable = true)

+-------------------+--------+-------+------------+--------+
|datesold           |postcode|price  |propertyType|bedrooms|
+-------------------+--------+-------+------------+--------+
|2007-02-07 00:00:00|2607    |525000 |house       |4       |
|2007-02-27 00:00:00|2906    |290000 |house       |3       |
|2007-03-07 00:00:00|2905    |328000 |house       |3       |
|2007-03-09 00:00:00|2905    |380000 |house       |4       |
|2007-03-21 00:00:00|2906    |310000 |house       |3       |
|2007-04-04 00:00:00|2905    |465000 |house       |4       |
|2007-04-24 00:00:00|2607    |399000 |house       |3       |
|2007-04-30 00:00:00|2606    |1530000|house       |4       |
|2007-05-24 00:00:00|2902    |359000 |house       |3       |
|2007-05-25 00:00:00|2906    |320000 |house       |3      

In [33]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

In [34]:
from pyspark.sql import Window

## Задание 1
Добавьте к таблице следующие поля:
*  Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode)
*  Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode)
*  Стоимость последнего проданного дома до текущего


In [8]:
# когда уже выполнила задание, то возник вопрос, относится ли юнит к понятию "домов" в задании?
# поскольку юнит тоже модет быть домом, просто на несколько семей.
# если юнит не входит в дома, то первой нужно делать группировку по propertyType

In [35]:
#Средняя стомость 10 проданных домов до текущего в том же районе
windSpec = Window()\
    .partitionBy('postcode')\
    .orderBy('datesold')\
    .rowsBetween(Window.currentRow - 11, Window.currentRow - 1)

df1 = df.withColumn('avg-10', F.avg('price').over(windSpec))
df1.show()

+-------------------+--------+-------+------------+--------+-----------------+
|           datesold|postcode|  price|propertyType|bedrooms|           avg-10|
+-------------------+--------+-------+------------+--------+-----------------+
|2007-07-08 00:00:00|    2600| 327000|       house|       1|             null|
|2007-08-16 00:00:00|    2600| 790000|       house|       4|         327000.0|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|         558500.0|
|2008-01-21 00:00:00|    2600| 315000|        unit|       1|647333.3333333334|
|2008-04-24 00:00:00|    2600| 292500|       house|       1|         564250.0|
|2008-05-30 00:00:00|    2600| 329000|        unit|       2|         509900.0|
|2008-06-19 00:00:00|    2600| 765000|       house|       5|         479750.0|
|2008-07-29 00:00:00|    2600| 927000|       house|       4|         520500.0|
|2008-09-02 00:00:00|    2600|1380000|       house|       5|         571312.5|
|2008-09-08 00:00:00|    2600| 740000|       house| 

In [36]:
#Средняя стомость 10 проданных домов после текущего в том же районе
windSpec = Window()\
    .partitionBy('postcode')\
    .orderBy('datesold')\
    .rowsBetween(Window.currentRow +1, Window.currentRow +11)

df1 = df1.withColumn('avg+10', F.avg('price').over(windSpec))
df1.show()

+-------------------+--------+-------+------------+--------+-----------------+-----------------+
|           datesold|postcode|  price|propertyType|bedrooms|           avg-10|           avg+10|
+-------------------+--------+-------+------------+--------+-----------------+-----------------+
|2007-07-08 00:00:00|    2600| 327000|       house|       1|             null|706681.8181818182|
|2007-08-16 00:00:00|    2600| 790000|       house|       4|         327000.0|692590.9090909091|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|         558500.0|703954.5454545454|
|2008-01-21 00:00:00|    2600| 315000|        unit|       1|647333.3333333334|741681.8181818182|
|2008-04-24 00:00:00|    2600| 292500|       house|       1|         564250.0|792818.1818181818|
|2008-05-30 00:00:00|    2600| 329000|        unit|       2|         509900.0|859045.4545454546|
|2008-06-19 00:00:00|    2600| 765000|       house|       5|         479750.0|816772.7272727273|
|2008-07-29 00:00:00|    2600|

In [37]:
# Стоимость последнего проданного дома до текущего
windSpec2 = Window()\
    .partitionBy('postcode')\
    .orderBy('datesold')\
    .rowsBetween(Window.unboundedPreceding, Window.currentRow - 1)

df1 = df1.withColumn('prev_price', F.last('price').over(windSpec2))
df1.show()

+-------------------+--------+-------+------------+--------+-----------------+-----------------+----------+
|           datesold|postcode|  price|propertyType|bedrooms|           avg-10|           avg+10|prev_price|
+-------------------+--------+-------+------------+--------+-----------------+-----------------+----------+
|2007-07-08 00:00:00|    2600| 327000|       house|       1|             null|706681.8181818182|      null|
|2007-08-16 00:00:00|    2600| 790000|       house|       4|         327000.0|692590.9090909091|    327000|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|         558500.0|703954.5454545454|    790000|
|2008-01-21 00:00:00|    2600| 315000|        unit|       1|647333.3333333334|741681.8181818182|    825000|
|2008-04-24 00:00:00|    2600| 292500|       house|       1|         564250.0|792818.1818181818|    315000|
|2008-05-30 00:00:00|    2600| 329000|        unit|       2|         509900.0|859045.4545454546|    292500|
|2008-06-19 00:00:00|    260

## Задание 2
В итоге у вас таблица с колонками (или нечто похожее):
*   price
*   Среднегодовая цена
*  Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode) (1 балл)
*  Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode) (1 балл)
*  Стоимость последнего проданного дома до текущего ((1 балл)
*  и др.

Посчитайте кол-во уникальных значений в каждой строчке (unique(row)) (ипользуйте udf). Попробуйте сделать то же самое используя pandas udf.

In [38]:
df1.show(3)

+-------------------+--------+------+------------+--------+--------+-----------------+----------+
|           datesold|postcode| price|propertyType|bedrooms|  avg-10|           avg+10|prev_price|
+-------------------+--------+------+------------+--------+--------+-----------------+----------+
|2007-07-08 00:00:00|    2600|327000|       house|       1|    null|706681.8181818182|      null|
|2007-08-16 00:00:00|    2600|790000|       house|       4|327000.0|692590.9090909091|    327000|
|2007-12-05 00:00:00|    2600|825000|       house|       3|558500.0|703954.5454545454|    790000|
+-------------------+--------+------+------------+--------+--------+-----------------+----------+
only showing top 3 rows



In [39]:
# udf

def f(row):
  return len(set(row))

f = F.udf(f, returnType=IntegerType())

df1.withColumn('value', f(F.array(df1['*']))).show()

+-------------------+--------+-------+------------+--------+-----------------+-----------------+----------+-----+
|           datesold|postcode|  price|propertyType|bedrooms|           avg-10|           avg+10|prev_price|value|
+-------------------+--------+-------+------------+--------+-----------------+-----------------+----------+-----+
|2007-07-08 00:00:00|    2600| 327000|       house|       1|             null|706681.8181818182|      null|    7|
|2007-08-16 00:00:00|    2600| 790000|       house|       4|         327000.0|692590.9090909091|    327000|    8|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|         558500.0|703954.5454545454|    790000|    8|
|2008-01-21 00:00:00|    2600| 315000|        unit|       1|647333.3333333334|741681.8181818182|    825000|    8|
|2008-04-24 00:00:00|    2600| 292500|       house|       1|         564250.0|792818.1818181818|    315000|    8|
|2008-05-30 00:00:00|    2600| 329000|        unit|       2|         509900.0|859045.454

In [23]:
import pandas as pd

In [41]:
# pandas udf

#PyArrow
spark.conf.set('sql.execution.arrow.pyspark.enable','true')

%timeit 
df1.toPandas()

def pandas_f(v):
    len(set(row))

df1.withColumn('value', f(F.array(df1['*']))).show()

+-------------------+--------+-------+------------+--------+-----------------+-----------------+----------+-----+
|           datesold|postcode|  price|propertyType|bedrooms|           avg-10|           avg+10|prev_price|value|
+-------------------+--------+-------+------------+--------+-----------------+-----------------+----------+-----+
|2007-07-08 00:00:00|    2600| 327000|       house|       1|             null|706681.8181818182|      null|    7|
|2007-08-16 00:00:00|    2600| 790000|       house|       4|         327000.0|692590.9090909091|    327000|    8|
|2007-12-05 00:00:00|    2600| 825000|       house|       3|         558500.0|703954.5454545454|    790000|    8|
|2008-01-21 00:00:00|    2600| 315000|        unit|       1|647333.3333333334|741681.8181818182|    825000|    8|
|2008-04-24 00:00:00|    2600| 292500|       house|       1|         564250.0|792818.1818181818|    315000|    8|
|2008-05-30 00:00:00|    2600| 329000|        unit|       2|         509900.0|859045.454

# Задание 3
SQL like case when или if elif else

Создайте колонку, в которой в которой будет отображаться "+", "-" или "=", если "Средняя стомость 10 проданных домов до текущего в том же районе" больше, меньше или равно "Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode)", соотвественно.

Если одно из полей Null, запишите в эту колонку "Нет данных"

In [40]:
from pyspark.sql.functions import when

#df.select(df.name, F.when(df.age > 4, 1).when(df.age < 3, -1).otherwise(0)).show()

df1.select(df1['datesold'], df1['postcode'], df1['price'], df1['avg-10'], df1['avg+10']\
           , F.when(df1['avg-10'] > df1['avg+10'],'+')\
           .when(df1['avg-10'] < df1['avg+10'],'-')\
           .when(df1['avg-10'] == df1['avg+10'],'=')\
           .otherwise('No')\
           .alias('comparison'))\
           .show()

+-------------------+--------+-------+-----------------+-----------------+----------+
|           datesold|postcode|  price|           avg-10|           avg+10|comparison|
+-------------------+--------+-------+-----------------+-----------------+----------+
|2007-07-08 00:00:00|    2600| 327000|             null|706681.8181818182|        No|
|2007-08-16 00:00:00|    2600| 790000|         327000.0|692590.9090909091|         -|
|2007-12-05 00:00:00|    2600| 825000|         558500.0|703954.5454545454|         -|
|2008-01-21 00:00:00|    2600| 315000|647333.3333333334|741681.8181818182|         -|
|2008-04-24 00:00:00|    2600| 292500|         564250.0|792818.1818181818|         -|
|2008-05-30 00:00:00|    2600| 329000|         509900.0|859045.4545454546|         -|
|2008-06-19 00:00:00|    2600| 765000|         479750.0|816772.7272727273|         -|
|2008-07-29 00:00:00|    2600| 927000|         520500.0|775681.8181818182|         -|
|2008-09-02 00:00:00|    2600|1380000|         571312.